In [1]:
import pandas as pd
from pandas import DataFrame
import csv

In [2]:
# reading in the csv file
original = pd.read_csv('CleanedGunViolenceData.csv')

In [3]:
original.head()

,date,state,city_or_county,n_killed,n_injured,congressional_district,gun_type,incident_characteristics,latitude,location_description,...,female participants,male participants,arrested,unharmed,unharmed/arrested,victims,subject/suspect,children,teens,adults
0,1/1/2014,Kentucky,Lexington,0,1,6.0,NaN,Shot - Wounded/Injured,38.0507,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1/1/2014,Kentucky,Cynthiana,0,1,4.0,0::Unknown,Shot - Wounded/Injured||Accidental Shooting||A...,38.4333,NaN,...,0.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0
2,1/1/2014,Kentucky,Louisville,0,1,3.0,NaN,Shot - Wounded/Injured,38.2501,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1/1/2014,Virginia,Falls Church,0,0,8.0,NaN,Armed robbery with injury/death and/or evidenc...,38.8847,NaN,...,NaN,NaN,0.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN
4,1/1/2014,Oklahoma,Lawton,0,0,4.0,NaN,Shots Fired - No Injuries,34.6347,NaN,...,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0


In [5]:
from sklearn import linear_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data_time = np.asarray(['2017-05-24', '2017-05-25', '2017-05-26',
                        '2017-05-27', '2017-05-28', '2017-05-29',
                        '2017-05-30', '2017-05-31', '2017-06-01',
                        '2017-06-02', '2017-06-03', '2017-06-04',
                        '2017-06-05', '2017-06-06', '2017-06-07',
                        '2017-06-08', '2017-06-09', '2017-06-10',
                        '2017-06-11', '2017-06-12', '2017-06-13',
                        '2017-06-14', '2017-06-15', '2017-06-16',
                        '2017-06-17', '2017-06-18', '2017-06-19',
                        '2017-06-20', '2017-06-21'])
data_count = np.asarray([300.000, 301.000, 302.000, 303.000, 304.000,
                         305.000, 306.000, 307.000, 308.000, 309.000,
                         310.000, 311.000, 312.000, 230.367, 269.032,
                         258.867, 221.645, 222.323, 212.357, 198.516,
                         230.133, 243.903, 244.320, 207.451, 192.710,
                         212.033, 216.677, 222.333, 208.710])

df = pd.DataFrame({'time': data_time, 'count': data_count})
df.time = pd.to_datetime(df.time)

regr = linear_model.LinearRegression()
regr.fit(df.time.values.reshape(-1, 1), df['count'].reshape(-1, 1))

# Make predictions using the testing set
y_pred = regr.predict(df.time.values.astype(float).reshape(-1, 1))
df['pred'] = y_pred

ax = df.plot(x='time', y='count', color='black', style='.')
df.plot(x='time', y='pred', color='orange', linewidth=3, ax=ax, alpha=0.5)
ax.set_title('My Title')
ax.set_xlabel('Date')
ax.set_ylabel('Metric')

plt.show()


AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
%pyspark
# Sample DataFrame with repeating words
GVdataframe = spark.original['incident_characteristics']
GVdataframe.show()

In [ ]:
%pyspark
# Tokenize the words
tokenizer = Tokenizer(inputCol="incident_characteristics", outputCol="tokens")
GVwordsData = tokenizer.transform(GVdataframe)
GVwordsData.show()

In [ ]:
%pyspark
# Run the hashing term frequency
hashing = HashingTF(inputCol="tokens", outputCol="hashedValues", numFeatures=pow(2,4))

# Transform into a DF
hashed_df = hashing.transform(GVwordsData)

In [ ]:
%pyspark
# Display new DataFrame
hashed_df.show(truncate=False)

In [ ]:
%pyspark
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [ ]:
%pyspark
# Display the DataFrame
rescaledData.select("incident_characteristics", "features").show(truncate=False)